In [33]:
import pandas as pd
import random
import sqlite3

#### Crear lista de calles aleatorias

In [16]:
# se crea la lista de posibles provincias
provinces = ["Buenos Aires", "Catamarca", "Chaco", "Chubut", 
             "Cordoba", "Corrientes", "Entre Rios", "Formosa", 
             "Jujuy", "La Pampa", "La Rioja", "Mendoza", "Misiones", "Neuquen",
              "Rio Negro", "Salta", "San Juan", "San Luis", "Santa Cruz",
              "Santa Fe", "Santiago del Estero", "Tierra del fuego", "Tucuman"]

In [3]:
# Se obtienen las localidades de cada provincia
location_dict = {}
for province in provinces:

    formatted_province = province.replace(" ", "").lower()
    link = f"https://www.citypopulation.de/php/argentina-{formatted_province}_s.php"

    df = pd.read_html(link)[0]
    locations = df["Nombre"].values.tolist()

    location_dict[province] = locations

In [7]:
# se obtienen ejemplos de calles de Argentina
streets = pd.read_html("https://es.wikipedia.org/wiki/Anexo:Calles_de_la_ciudad_de_Buenos_Aires")[0]
streets = streets["Nombre"].values.tolist()

In [11]:
# se generan datos aleatorios de alturas de calles
heights = list(range(100, 9999))

#### Se generan las direcciones para agregar a la base de datos

In [17]:
provinces_selection = random.choices(provinces, k=1100, weights = [0.2, 0.025, 0.025, 0.025, 0.15, 0.025, 0.05, 0.025, 0.05, 0.025, 
                                                                   0.025, 0.05, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.05, 
                                                                   0.025, 0.025, 0.05])

In [23]:
location_selections = []
for i, j in enumerate(provinces_selection):
    
    location_selection = random.choice(location_dict[provinces_selection[i]])
    location_selections.append(location_selection)

In [25]:
streets_selection = random.choices(streets, k=1100)

In [26]:
heights_selection = random.choices(heights, k=1100)

#### Sumando todos los puntos

In [39]:
directions = [ x + ", "  + y +  ", " + z + " " + str(m) for x, y, z, m in zip(provinces_selection, location_selections, streets_selection, heights_selection)]

In [41]:
directions = pd.DataFrame(directions, columns=["direccion"])

### Se agregan las direcciones a la base de datos para los clientes, empleados y sucursales

In [10]:
con = sqlite3.connect("..\database\itbank.db")

### Se agrega la columna de dirección a la base de datos de clientes

In [14]:
clientes = pd.read_sql("SELECT * FROM cliente", con=con)

In [19]:
clientes["directions"] = directions["direccion"].head(500)

In [21]:
### overwrite the clientes dataframe with the new column
clientes.to_sql("cliente", con=con, if_exists="replace")

### Se agrega la columna de dirección a la base de empleados y sucursales

In [34]:
empleados = pd.read_sql("SELECT * FROM empleado", con=con)

In [38]:
empleados["directions"] = directions["direccion"][500:1000].values.tolist()

In [40]:
empleados.to_sql("empleado", con=con, if_exists="replace")

--------

In [41]:
sucursal = pd.read_sql("SELECT * FROM sucursal", con=con)

In [42]:
sucursal["directions"] = directions["direccion"][1000:1500].values.tolist()

In [43]:
sucursal.to_sql("sucursal", con=con, if_exists="replace")

#### Se cierra la conexión con la base de datos (importante para evitar errores futuros)

In [ ]:
con.close()